In [25]:
import os
from PyPDF2 import PdfReader
import re
import csv
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# Function to extract text and metadata from a PDF file
def extract_pdf_info(file_path):
    pdf_reader = PdfReader(file_path)
    text = ""
    metadata = {
        'name': '',
        'email': '',
        'phone': '',
        'linkedin': ''
    }

    for page_num in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page_num].extract_text()

    # Extract metadata using regular expressions
    name_pattern = r'Name: (.*?)\n'
    email_pattern = r'Email: (.*?)\n'
    phone_pattern = r'Phone: (\d{3}-\d{3}-\d{4})\n'
    linkedin_pattern = r'LinkedIn: (.*?)\n'

    matches = re.findall(name_pattern, text)
    if matches:
        metadata['name'] = matches[0].strip()

    matches = re.findall(email_pattern, text)
    if matches:
        metadata['email'] = matches[0].strip()

    matches = re.findall(phone_pattern, text)
    if matches:
        metadata['phone'] = matches[0].strip()

    matches = re.findall(linkedin_pattern, text)
    if matches:
        metadata['linkedin'] = matches[0].strip()

    return text, metadata

# Function to generate a summary of the candidate's information
def generate_summary(text):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, 1)  # You can change the number of sentences in the summary
    summary_text = ""
    for sentence in summary:
        summary_text += str(sentence) + " "
    return summary_text

# Function to write the output to a CSV file
def write_output_to_csv(file_name, metadata, summary):
    output_file = 'output.csv'
    fieldnames = ['file_name', 'name', 'email', 'phone', 'linkedin', 'summary']

    if os.path.exists(output_file):
        with open(output_file, 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow({'file_name': file_name, 'name': metadata['name'], 'email': metadata['email'], 'phone': metadata['phone'], 'linkedin': metadata['linkedin'], 'summary': summary})
    else:
        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerow({'file_name': file_name, 'name': metadata['name'], 'email': metadata['email'], 'phone': metadata['phone'], 'linkedin': metadata['linkedin'], 'summary': summary})

# Main function to process multiple PDF files
def process_pdf_files(file_paths):
    for file_path in file_paths:
        if file_path.endswith('.pdf'):
            text, metadata = extract_pdf_info(file_path)
            summary = generate_summary(text)
            write_output_to_csv(os.path.basename(file_path), metadata, summary)

# Example usage
filer_path = '/content'
file_names = ['swastik.pdf', 'Resume(1).pdf']
file_paths = [os.path.join(folder_path, file_name) for file_name in file_names]
process_pdf_files(file_paths)

